In [2]:
import pandas as pd
import sys
sys.path.append('../word-embeddings-benchmark')

from web.datasets.similarity import fetch_MTurk, fetch_MEN, fetch_WS353, fetch_RG65, fetch_RW, fetch_SimLex999, fetch_TR9856
from web.datasets.analogy import fetch_google_analogy, fetch_msr_analogy, fetch_wordrep, fetch_semeval_2012_2

#Important for converting embeddings to managable format
from web.embedding import Embedding
from web.embeddings import fetch_HPCA, fetch_morphoRNNLM
from web.evaluate import evaluate_similarity, evaluate_analogy, evaluate_on_WordRep, evaluate_on_semeval_2012_2

## Embeddings
All embeddings are used without normalization

### GloVe:
400,000 words, 6B tokens with 50, 100, 200, 300 dimensions

### Fast Text
wiki-news 16B tokens with 300 dimensions

### LexVec
Note: LexVec Common Crawl 58B tokens , 300 dimensions was 5GB. Hence uable to load onto memory
Word + Context Vectors, 7B tokens, 300 dimensions

### ConceptNet Number batch
300 dimensions

### Google News
300 dimensions

### PDC
300 dimensions

### HDC
300 dimensions

In [4]:
#Load time is about 5-10 mins
glove = Embedding.from_glove('../Data/Embeddings/GloVe/glove.6B.300d.txt', 400000, 300)

We ignored line number 400000 because of errors in parsing
index 400000 is out of bounds for axis 0 with size 400000


In [7]:
#Load time is about 10-15 mins
fast_text = Embedding.from_word2vec('../Data/Embeddings/FastText/wiki-news-300d-1M.vec')

In [8]:
#Load time is about 10 mins
lex_vec = Embedding.from_word2vec('../Data/Embeddings/LexVec/lexvec.enwiki+newscrawl.300d.W+C.pos.vectors')

In [9]:
#load time is about 10 mins
concept_net = Embedding.from_word2vec('../Data/Embeddings/ConceptNet/numberbatch-en.txt')

In [10]:
#Google news
google_news = Embedding.from_word2vec('../Data/Embeddings/GoogleNews/GoogleNews-vectors-negative300.bin', binary=True)

In [11]:
#PDC
pdc = Embedding.from_word2vec('../Data/Embeddings/PDC/wikicorp.201004-pdc-iter-20-alpha-0.05-window-10-dim-300-neg-10-subsample-0.0001.txt')

In [12]:
#HDC
hdc = Embedding.from_word2vec('../Data/Embeddings/HDC/wikicorp.201004-hdc-iter-20-alpha-0.025-window-10-dim-300-neg-10-subsample-0.0001.txt')

In [7]:
#morpho RNN HSMN
# hpca = Embedding.from_word2vec('../Data/Embeddings/HPCA/words.bin', binary=True)
# fetch_morphoRNNL("HSMN")

## Similarity Task

MTurk <br/>
Evaluated by 10 people on a scale of  1 - 5. Scores are multipled by 2

MEN <br/>
Pairs: 3000
Scores range : 
Natural form not lemmatized form. Scores were rescaled to stay within 0 - 10 range

WS353 <br/>
Pairs: 353
Scores range : 0 - 10

RG65 <br/>
Pairs: 65
Scores range : 0 - 4. Scaled by factor 10/4

Rare Words (RW) <br/>
Pairs: 2034
Scores range : 0 - 10

SimLex 999 (SimLex) <br/>
Pairs: 999
Scores range : 0 - 10

TR9856 <br/>
Pairs: 9856
Scores range : 0 - 1


In [5]:
#This is for the cummulative results
results = pd.read_csv('../Data/Similarity/Results.csv')
results

,Dataset,Mturk,MEN,WS353,RG65,RW,SimLex,TR9856
0,Glove,0.6331,0.7374,0.5435,0.7695,0.3670,0.3710,0.0994
1,Fast Text,0.7022,0.7906,0.7332,0.8463,0.5134,0.4499,0.1642
2,LexVec,0.6480,0.7654,0.6381,0.7932,0.4467,0.3626,0.1312
3,ConceptNet,0.7188,0.8661,0.7682,0.9248,0.5686,0.6267,0.1321
4,GoogleNews,0.6805,0.7585,0.7000,0.7607,0.4977,0.4420,0.1813
5,PDC,0.6723,0.7726,0.7333,0.7901,0.4723,0.4269,0.2045
6,HDC,0.6576,0.7604,0.7166,0.9058,0.4634,0.4068,0.2056


In [20]:
#All Similiarty datasets
mturk = fetch_MTurk('../Data/Similarity/EN-TRUK.txt')
men = fetch_MEN('../Data/Similarity/EN-MEN-LEM.txt')
ws353 = fetch_WS353('../Data/Similarity/WS353_combined.csv')
rg65 = fetch_RG65('../Data/Similarity/EN-RG-65.txt')
rw = fetch_RW('../Data/Similarity/RW.txt')
simlex = fetch_SimLex999('../Data/Similarity/EN-SIM999.txt')
tr9856 = fetch_TR9856('../Data/Similarity/TR9856.csv')

datasets = {
    "mturk" : mturk,
    "men" : men,
    "ws353" : ws353,
    "rg65" : rg65,
    "rw" : rw,
    "simlex" : simlex,
    "tr9856" : tr9856
}

In [21]:
def compute_similarity(embedding):
    results_df = pd.DataFrame(columns=["Dataset", "Spearman Correlation value"])
    
    for key, value in datasets.items():
        corr_val = evaluate_similarity(embedding, value['X'], value['y'])
        results_df.loc[len(results_df)] = [key, corr_val]
        
    return results_df

In [24]:
#GloVe
compute_similarity(glove)

Missing 24 words. Will replace them with mean vector
Missing 260 words. Will replace them with mean vector
Missing 12079 words. Will replace them with mean vector


,Dataset,Spearman Correlation value
0,mturk,0.633182
1,men,0.737465
2,ws353,0.543524
3,rg65,0.769525
4,rw,0.367045
5,simlex,0.370500
6,tr9856,0.099480


In [23]:
#Fast Text
compute_similarity(fast_text)

Missing 2 words. Will replace them with mean vector
Missing 68 words. Will replace them with mean vector
Missing 12191 words. Will replace them with mean vector


,Dataset,Spearman Correlation value
0,mturk,0.702289
1,men,0.790633
2,ws353,0.733276
3,rg65,0.846259
4,rw,0.513497
5,simlex,0.449965
6,tr9856,0.164246


In [25]:
#Lex Vec
compute_similarity(lex_vec)

Missing 1 words. Will replace them with mean vector
Missing 24 words. Will replace them with mean vector
Missing 288 words. Will replace them with mean vector
Missing 1 words. Will replace them with mean vector
Missing 12087 words. Will replace them with mean vector


,Dataset,Spearman Correlation value
0,mturk,0.648025
1,men,0.765485
2,ws353,0.638120
3,rg65,0.793260
4,rw,0.446721
5,simlex,0.362670
6,tr9856,0.131299


In [26]:
#ConceptNet Number batch
compute_similarity(concept_net)

Missing 24 words. Will replace them with mean vector
Missing 204 words. Will replace them with mean vector
Missing 12142 words. Will replace them with mean vector


,Dataset,Spearman Correlation value
0,mturk,0.718810
1,men,0.866130
2,ws353,0.768220
3,rg65,0.924829
4,rw,0.568671
5,simlex,0.626787
6,tr9856,0.132052


In [39]:
#Google News
compute_similarity(google_news)

Missing 12 words. Will replace them with mean vector
Missing 55 words. Will replace them with mean vector
Missing 216 words. Will replace them with mean vector
Missing 12282 words. Will replace them with mean vector


,Dataset,Spearman Correlation value
0,mturk,0.680567
1,men,0.758586
2,ws353,0.700017
3,rg65,0.760783
4,rw,0.497720
5,simlex,0.441966
6,tr9856,0.181381


In [23]:
#PDC
compute_similarity(pdc)

Missing 1 words. Will replace them with mean vector
Missing 24 words. Will replace them with mean vector
Missing 180 words. Will replace them with mean vector
Missing 12091 words. Will replace them with mean vector


,Dataset,Spearman Correlation value
0,mturk,0.672333
1,men,0.772648
2,ws353,0.733290
3,rg65,0.790069
4,rw,0.472361
5,simlex,0.426882
6,tr9856,0.204566


In [24]:
#HDC
compute_similarity(hdc)

Missing 1 words. Will replace them with mean vector
Missing 24 words. Will replace them with mean vector
Missing 180 words. Will replace them with mean vector
Missing 12091 words. Will replace them with mean vector


,Dataset,Spearman Correlation value
0,mturk,0.657670
1,men,0.760335
2,ws353,0.716590
3,rg65,0.805805
4,rw,0.463469
5,simlex,0.406832
6,tr9856,0.205622


In [8]:
#morpho RNN HSMN
# compute_similarity(hsmn)

## Analogy

Experiments (In terms of overall accuracy on all categories):<br/>
Results are provided with add rather than mul <br/>
<br/>

### Google Analogy

In [6]:
annalogy_results = pd.read_csv('../Data/Analogy/Results.csv')
annalogy_results

,Dataset,Google Analogy,MSR,SEMEVAL 2012 Task 2
0,Glove,65.10,51.40,16.91
1,Fast Text,9.28,17.85,22.73
2,LexVec,60.40,37.08,18.15
3,ConceptNet,31.94,43.95,25.65
4,GoogleNews,1.23,0.60,22.39
5,PDC,38.10,26.53,18.71
6,HDC,28.90,13.95,18.78


In [5]:
def compute_analogy(embedding, dataset):
    return evaluate_analogy(embedding, dataset['X'], dataset['y'], category=dataset['category'])

In [27]:
google_analogy = fetch_google_analogy('../Data/Analogy/EN-GOOGLE.txt')

In [31]:
#GloVe
results = compute_analogy(glove, google_analogy)
results

,accuracy,correct,count
all,0.650993,12723,19544
gram8-plural,0.706456,941,1332
gram5-present-participle,0.528409,558,1056
currency,0.189376,164,866
capital-world,0.912246,4127,4524
gram4-superlative,0.628342,705,1122
city-in-state,0.602351,1486,2467
gram3-comparative,0.804805,1072,1332
gram9-plural-verbs,0.498851,434,870
gram2-opposite,0.209360,170,812


In [32]:
#Fast Text
results = compute_analogy(fast_text, google_analogy)
results

Missing 4667 words. Will replace them with mean vector


,accuracy,correct,count
all,0.092867,1815,19544
gram8-plural,0.000000,0,1332
gram5-present-participle,0.171402,181,1056
currency,0.000000,0,866
capital-world,0.000000,0,4524
gram4-superlative,0.291444,327,1122
city-in-state,0.000000,0,2467
gram3-comparative,0.444444,592,1332
gram9-plural-verbs,0.150575,131,870
gram2-opposite,0.113300,92,812


In [33]:
#Lex Vec
results = compute_analogy(lex_vec, google_analogy)
results

,accuracy,correct,count
all,0.604022,11805,19544
gram8-plural,0.560811,747,1332
gram5-present-participle,0.366477,387,1056
currency,0.271363,235,866
capital-world,0.862069,3900,4524
gram4-superlative,0.491979,552,1122
city-in-state,0.630726,1556,2467
gram3-comparative,0.755255,1006,1332
gram9-plural-verbs,0.391954,341,870
gram2-opposite,0.200739,163,812


In [34]:
#ConceptNet Number batch
results = compute_analogy(concept_net, google_analogy)
results

,accuracy,correct,count
all,0.319382,6242,19544
gram8-plural,0.162162,216,1332
gram5-present-participle,0.242424,256,1056
currency,0.128176,111,866
capital-world,0.388815,1759,4524
gram4-superlative,0.680036,763,1122
city-in-state,0.158087,390,2467
gram3-comparative,0.554054,738,1332
gram9-plural-verbs,0.243678,212,870
gram2-opposite,0.096059,78,812


In [40]:
#Google News
results = compute_analogy(google_news, google_analogy)
results

Missing 9402 words. Will replace them with mean vector
../word-embeddings-benchmark\web\analogy.py:105: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  A, B, C = np.vstack(w.get(word, mean_vector) for word in X_b[:, 0]), \
../word-embeddings-benchmark\web\analogy.py:106: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  np.vstack(w.get(word, mean_vector) for word in X_b[:, 1]), \
../word-embeddings-benchmark\web\analogy.py:107: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  np.vstack(w.get(wo

,accuracy,correct,count
all,0.012331,241,19544
gram8-plural,0.049550,66,1332
gram5-present-participle,0.024621,26,1056
currency,0.006928,6,866
capital-world,0.001326,6,4524
gram4-superlative,0.001783,2,1122
city-in-state,0.021484,53,2467
gram3-comparative,0.008258,11,1332
gram9-plural-verbs,0.045977,40,870
gram2-opposite,0.000000,0,812


In [42]:
#PDC
results = compute_analogy(pdc, google_analogy)
results

,accuracy,correct,count
all,0.380833,7443,19544
gram8-plural,0.491742,655,1332
gram5-present-participle,0.263258,278,1056
currency,0.027714,24,866
capital-world,0.478338,2164,4524
gram4-superlative,0.395722,444,1122
city-in-state,0.475882,1174,2467
gram3-comparative,0.530030,706,1332
gram9-plural-verbs,0.520690,453,870
gram2-opposite,0.221675,180,812


In [44]:
#HDC
results = compute_analogy(hdc, google_analogy)
results

../word-embeddings-benchmark\web\analogy.py:105: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  A, B, C = np.vstack(w.get(word, mean_vector) for word in X_b[:, 0]), \
../word-embeddings-benchmark\web\analogy.py:106: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  np.vstack(w.get(word, mean_vector) for word in X_b[:, 1]), \
../word-embeddings-benchmark\web\analogy.py:107: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  np.vstack(w.get(word, mean_vector) for word in X_b[:, 2])


,accuracy,correct,count
all,0.288682,5642,19544
gram8-plural,0.205706,274,1332
gram5-present-participle,0.155303,164,1056
currency,0.124711,108,866
capital-world,0.403846,1827,4524
gram4-superlative,0.205882,231,1122
city-in-state,0.407377,1005,2467
gram3-comparative,0.292793,390,1332
gram9-plural-verbs,0.414943,361,870
gram2-opposite,0.158867,129,812


## MSR

In [10]:
msr = fetch_msr_analogy('../Data/Analogy/EN-MSR.txt')

In [13]:
#GloVe
results = compute_analogy(glove, msr)
results

Missing 164 words. Will replace them with mean vector
../word-embeddings-benchmark\web\analogy.py:105: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  A, B, C = np.vstack(w.get(word, mean_vector) for word in X_b[:, 0]), \
../word-embeddings-benchmark\web\analogy.py:106: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  np.vstack(w.get(word, mean_vector) for word in X_b[:, 1]), \
../word-embeddings-benchmark\web\analogy.py:107: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  np.vstack(w.get(wor

,accuracy,correct,count
all,0.514125,4113,8000
jjs_jj,0.350000,175,500
jj_jjs,0.478000,239,500
nnpos_nn,0.456000,228,500
jjs_jjr,0.574000,287,500
jj_jjr,0.596000,298,500
nns_nn,0.518000,259,500
vb_vbz,0.656000,328,500
vbz_vb,0.780000,390,500
nn_nns,0.680000,340,500


In [14]:
#Fast Text
results = compute_analogy(fast_text, msr)
results

,accuracy,correct,count
all,0.1785,1428,8000
jjs_jj,0.0900,45,500
jj_jjs,0.1860,93,500
nnpos_nn,0.0000,0,500
jjs_jjr,0.2500,125,500
jj_jjr,0.3120,156,500
nns_nn,0.0000,0,500
vb_vbz,0.2160,108,500
vbz_vb,0.4180,209,500
nn_nns,0.0000,0,500


In [15]:
#Lex Vec
results = compute_analogy(lex_vec, msr)
results

Missing 131 words. Will replace them with mean vector


,accuracy,correct,count
all,0.37075,2966,8000
jjs_jj,0.22200,111,500
jj_jjs,0.45000,225,500
nnpos_nn,0.27600,138,500
jjs_jjr,0.47200,236,500
jj_jjr,0.54000,270,500
nns_nn,0.32200,161,500
vb_vbz,0.40800,204,500
vbz_vb,0.50400,252,500
nn_nns,0.51000,255,500


In [16]:
#Concept Net
results = compute_analogy(concept_net, msr)
results

Missing 302 words. Will replace them with mean vector


,accuracy,correct,count
all,0.4395,3516,8000
jjs_jj,0.6020,301,500
jj_jjs,0.5220,261,500
nnpos_nn,0.3360,168,500
jjs_jjr,0.7900,395,500
jj_jjr,0.4300,215,500
nns_nn,0.3900,195,500
vb_vbz,0.3060,153,500
vbz_vb,0.7040,352,500
nn_nns,0.0480,24,500


In [17]:
#Google News
results = compute_analogy(google_news, msr)
results

,accuracy,correct,count
all,0.006625,53,8000
jjs_jj,0.004000,2,500
jj_jjs,0.010000,5,500
nnpos_nn,0.004000,2,500
jjs_jjr,0.012000,6,500
jj_jjr,0.022000,11,500
nns_nn,0.008000,4,500
vb_vbz,0.006000,3,500
vbz_vb,0.000000,0,500
nn_nns,0.004000,2,500


In [18]:
#PDC
results = compute_analogy(pdc, msr)
results

Missing 241 words. Will replace them with mean vector


,accuracy,correct,count
all,0.26525,2122,8000
jjs_jj,0.06800,34,500
jj_jjs,0.26400,132,500
nnpos_nn,0.18000,90,500
jjs_jjr,0.25400,127,500
jj_jjr,0.28000,140,500
nns_nn,0.19600,98,500
vb_vbz,0.65200,326,500
vbz_vb,0.39000,195,500
nn_nns,0.29800,149,500


In [19]:
#HDC
results = compute_analogy(hdc, msr)
results

Missing 241 words. Will replace them with mean vector


,accuracy,correct,count
all,0.1395,1116,8000
jjs_jj,0.0260,13,500
jj_jjs,0.2220,111,500
nnpos_nn,0.1000,50,500
jjs_jjr,0.1400,70,500
jj_jjr,0.1920,96,500
nns_nn,0.0920,46,500
vb_vbz,0.3660,183,500
vbz_vb,0.1440,72,500
nn_nns,0.1460,73,500


## SEMEVAL 2012

In [2]:
semeval_2 = fetch_semeval_2012_2('../Data/Analogy/SemEval-2/', "all", "golden")

In [3]:
len(semeval_2['X'])

79

In [6]:
#GloVe
results = evaluate_on_semeval_2012_2('../Data/Analogy/SemEval-2/', "all", "golden", glove)
results

all                0.169063
REFERENCE          0.168442
NON-ATTRIBUTE      0.107258
CAUSE-PURPOSE      0.167494
CLASS-INCLUSION    0.167333
PART-WHOLE         0.125251
CASE RELATIONS     0.276244
SIMILAR            0.121736
CONTRAST           0.082778
ATTRIBUTE          0.236260
SPACE-TIME         0.231743
dtype: float64

In [13]:
#Fast Text
results = evaluate_on_semeval_2012_2('../Data/Analogy/SemEval-2/', "all", "golden", fast_text)
results

all                0.227285
REFERENCE          0.116462
NON-ATTRIBUTE      0.160227
CAUSE-PURPOSE      0.310239
CLASS-INCLUSION    0.266009
PART-WHOLE         0.169698
CASE RELATIONS     0.373906
SIMILAR            0.180387
CONTRAST           0.102036
ATTRIBUTE          0.320628
SPACE-TIME         0.258860
dtype: float64

In [14]:
#Lex Vec
results = evaluate_on_semeval_2012_2('../Data/Analogy/SemEval-2/', "all", "golden", lex_vec)
results

all                0.181464
REFERENCE          0.143208
NON-ATTRIBUTE      0.118441
CAUSE-PURPOSE      0.166339
CLASS-INCLUSION    0.263356
PART-WHOLE         0.138580
CASE RELATIONS     0.279870
SIMILAR            0.125687
CONTRAST           0.051032
ATTRIBUTE          0.263966
SPACE-TIME         0.274133
dtype: float64

In [15]:
#Concept Net
results = evaluate_on_semeval_2012_2('../Data/Analogy/SemEval-2/', "all", "golden", concept_net)
results

all                0.256531
REFERENCE          0.193799
NON-ATTRIBUTE      0.213108
CAUSE-PURPOSE      0.262324
CLASS-INCLUSION    0.330811
PART-WHOLE         0.176462
CASE RELATIONS     0.392471
SIMILAR            0.273852
CONTRAST           0.081094
ATTRIBUTE          0.350396
SPACE-TIME         0.305351
dtype: float64

In [16]:
#Google News
results = evaluate_on_semeval_2012_2('../Data/Analogy/SemEval-2/', "all", "golden", google_news)
results

all                0.223870
REFERENCE          0.130743
NON-ATTRIBUTE      0.127698
CAUSE-PURPOSE      0.277416
CLASS-INCLUSION    0.257192
PART-WHOLE         0.175357
CASE RELATIONS     0.390189
SIMILAR            0.216442
CONTRAST           0.075987
ATTRIBUTE          0.297229
SPACE-TIME         0.276094
dtype: float64

In [17]:
#PDC
results = evaluate_on_semeval_2012_2('../Data/Analogy/SemEval-2/', "all", "golden", pdc)
results

all                0.187168
REFERENCE          0.149982
NON-ATTRIBUTE      0.096745
CAUSE-PURPOSE      0.269406
CLASS-INCLUSION    0.226705
PART-WHOLE         0.162016
CASE RELATIONS     0.296438
SIMILAR            0.102000
CONTRAST           0.061526
ATTRIBUTE          0.278467
SPACE-TIME         0.224178
dtype: float64

In [18]:
#HDC
results = evaluate_on_semeval_2012_2('../Data/Analogy/SemEval-2/', "all", "golden", hdc)
results

all                0.187836
REFERENCE          0.125699
NON-ATTRIBUTE      0.105210
CAUSE-PURPOSE      0.260519
CLASS-INCLUSION    0.221783
PART-WHOLE         0.135990
CASE RELATIONS     0.322025
SIMILAR            0.092252
CONTRAST           0.091169
ATTRIBUTE          0.270137
SPACE-TIME         0.246160
dtype: float64

## WordRep

In [6]:
#Word Rep
word_rep = fetch_wordrep('../Data/Analogy/WordRep/', 0.005, 10)

In [7]:
len(word_rep['X'])

222

In [9]:
#GloVe
results = evaluate_on_WordRep('../Data/Analogy/WordRep/', 0.005, 10, glove)
results

In [ ]:
#Fast Text
results = evaluate_on_WordRep('../Data/Analogy/WordRep/', 0.4, 10, fast_text)
results

In [ ]:
#Lex Vec
results = evaluate_on_WordRep('../Data/Analogy/WordRep/', 0.4, 10, lex_vec)
results

In [ ]:
#Concept Net
results = evaluate_on_WordRep('../Data/Analogy/WordRep/', 0.4, 10, concept_new)
results

In [ ]:
#Google News
results = evaluate_on_WordRep('../Data/Analogy/WordRep/', 0.4, 10, google_news)
results

In [ ]:
#PDC
results = evaluate_on_WordRep('../Data/Analogy/WordRep/', 0.4, 10, pdc)
results

In [ ]:
#HDC
results = evaluate_on_WordRep('../Data/Analogy/WordRep/', 0.4, 10, hdc)
results

In [46]:
#ConceptNet
results = evaluate_analogy(concept_net, google_analogy['X'], google_analogy['y'], category=google_analogy['category'])
results 